<a href="https://colab.research.google.com/github/UndeadZed/OPT-MVP/blob/main/LLAMA3_1_text_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all essentials for the Project

In [ ]:
# solving the build problem by installing the missing scikit-build-core
!pip install scikit-build-core==0.9.0
# For download the models
!pip install huggingface_hub
#installing langchain for inferance
!pip -q install langchain
# installing Llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=61" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.62 --force-reinstall --upgrade --no-cache-dir --verbose --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 173.8 MB/s eta 0:00:00
  Running command Preparing metadata (pyproject

# Downloading the model

In [ ]:
from huggingface_hub import hf_hub_download

# model_name_or_path = "TheBloke/Tess-34B-v1.5b-GGUF"
# model_basename = "tess-34b-v1.5b.Q3_K_M.gguf" # the model is in gguf format

model_name_or_path = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF"
model_basename = "capybarahermes-2.5-mistral-7b.Q8_0.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

capybarahermes-2.5-mistral-7b.Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

# Making LLAMACPP-python LLM object

In [ ]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)


llama_model_loader: loaded meta data with 23 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--LLaMA-Pro-8B-Instruct-GGUF/snapshots/ebde3204ccf6ec8f75fa784d013dbde43923c847/llama-pro-8b-instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llam

## Maing infrences based in LLAMAcpp-python object

In [ ]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [ ]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)

print(response["choices"][0]["text"])


llama_print_timings:        load time =   10649.51 ms
llama_print_timings:      sample time =     145.70 ms /   256 runs   (    0.57 ms per token,  1757.00 tokens per second)
llama_print_timings: prompt eval time =   10649.32 ms /    39 tokens (  273.06 ms per token,     3.66 tokens per second)
llama_print_timings:        eval time =    9347.42 ms /   255 runs   (   36.66 ms per token,    27.28 tokens per second)
llama_print_timings:       total time =   20961.18 ms /   294 tokens


SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:
Here's an example of how to write a simple linear regression model using Python with the scikit-learn library:

```python
from sklearn.linear_model import LinearRegression
import numpy as np

# Create some sample data for demonstration purposes
X = np.array([1, 2, 3, 4]).reshape(-1, 1) # input features (independent variables)
y = np.array([5, 7, 9, 11]) # output values (dependent variable)

# Instantiate the linear regression model
model = LinearRegression()

# Fit the model to our data
model.fit(X, y)
```
In this code:
- We imported `LinearRegression` from scikit-learn's `linear_model` module and `numpy`. 
- Then we created a simple set of input features (independent variables), X, and an output variable (dependent variable), y.
- The reshape(-1, 1) method is used to change the shape of our array from [4] -> [-1]. It's necessary because linear 

## clearing llamacpp cache

In [ ]:
lcpp_llm.reset()
lcpp_llm.set_cache(None)
lcpp_llm = None
del lcpp_llm

# Making an LLM object with Langchain

In [ ]:
!pip -q install langchain
!pip -q install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download

# template = """
# {question}
#  """
# prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0.3,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--CapybaraHermes-2.5-Mistral-7B-GGUF/snapshots/8bea614edd9a2d5d9985a6e6c1ecc166261cacb8/capybarahermes-2.5-mistral-7b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = argilla_capybarahermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llam

In [ ]:
context_text = """
SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {question}

ASSISTANT:
"""

In [ ]:
input_text = """
satisfy
a
shipping
contract
,
a
total
of
at
least
60,000
square
feet
of
flooring
much
be
shipped
each
week
.
Due
to
a
labor
shortage
issue
,
no
more
than
50,000
square
feet
of
hardwood
and
30,000
square
feet
of
vinyl
can
be
produced
weekly
.
If
a
square
foot
of
hardwood
flooring
yields
a
profit
of
$
2.5
and
a
square
foot
of
vinyl
planks
produces
a
$
3
profit
,
how
many
of
each
type
of
flooring
should
be
made
weekly
to
maximize
the
company
's
profit
?

John
has
a
300
acre
berry
farm
on
which
to
plant
blueberries
and
raspberries
.
John
has
$
10000
to
spend
on
watering
and
575
days
worth
of
labor
available
.
For
each
acre
of
blueberries
,
6
days
worth
of
labor
and
$
22
in
watering
costs
is
required
.
For
each
acre
of
raspberries
,
3
days
worth
of
labor
and
$
25
in
watering
costs
is
required
.
The
profit
per
acre
of
blueberries
is
$
56
and
the
profit
per
acre
of
raspberries
is
$
75
.
Formulate
an
LP
problem
in
order
to
maximize
profit
.

An
electronics
store
wants
to
optimize
how
many
phones
and
laptops
are
enough
to
keep
in
inventory
.
A
phone
will
earn
the
store
$
120
in
profits
,
and
a
laptop
will
earn
$
40
.
A
phone
requires
1
sq
ft
of
floor
space
,
whereas
a
laptop
requires
4
sq
ft
.
In
total
,
400
sq
ft
of
floor
"""

In [ ]:
# llm_chain = LLMChain(prompt=prompt, llm=llm)

prompt = """ 'System: the following [unlabeled text (1)] is an example of an unlabled text followed by the same text after labeling [labeled text (1)],as you can see labeled text consists of 4 columns with the first being the unlabeled text while the other three consisting of symbols and labels write [labeled text (2)] based on the labels and format of the labeled text presented prior.'

## unlabeled text (1):

Cautious
Asset
Investment
has
a
total
of
$
150,000
to
manage
and
decides
to
invest
it
in
money
market
fund
,
which
yields
a
2
%
return
as
well
as
in
foreign
bonds
,
which
gives
and
average
rate
of
return
of
10.2
%
.
Internal
policies
require
PAI
to
diversify
the
asset
allocation
so
that
the
minimum
investment
in
money
market
fund
is
40
%
of
the
total
investment
.
Due
to
the
risk
of
default
of
foreign
countries
,
no
more
than
40
%
of
the
total
investment
should
be
allocated
to
foreign
bonds
.
How
much
should
the
Cautious
Asset
Investment
allocate
in
each
asset
so
as
to
maximize
its
average
return
?

## labeled text (1):
Cautious	_	_	O
Asset	_	_	O
Investment	_	_	O
has	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
$	_	_	O
150,000	_	_	B-LIMIT
to	_	_	O
manage	_	_	O
and	_	_	O
decides	_	_	O
to	_	_	O
invest	_	_	O
it	_	_	O
in	_	_	O
money	_	_	B-VAR
market	_	_	I-VAR
fund	_	_	I-VAR
,	_	_	O
which	_	_	O
yields	_	_	O
a	_	_	O
2	_	_	B-PARAM
%	_	_	I-PARAM
return	_	_	B-OBJ_NAME
as	_	_	O
well	_	_	O
as	_	_	O
in	_	_	O
foreign	_	_	B-VAR
bonds	_	_	I-VAR
,	_	_	O
which	_	_	O
gives	_	_	O
and	_	_	O
average	_	_	O
rate	_	_	O
of	_	_	O
return	_	_	B-OBJ_NAME
of	_	_	O
10.2	_	_	B-PARAM
%	_	_	O
.	_	_	O
Internal	_	_	O
policies	_	_	O
require	_	_	O
PAI	_	_	O
to	_	_	O
diversify	_	_	O
the	_	_	O
asset	_	_	O
allocation	_	_	O
so	_	_	O
that	_	_	O
the	_	_	O
minimum	_	_	B-CONST_DIR
investment	_	_	O
in	_	_	O
money	_	_	B-VAR
market	_	_	I-VAR
fund	_	_	I-VAR
is	_	_	O
40	_	_	B-LIMIT
%	_	_	I-LIMIT
of	_	_	O
the	_	_	O
total	_	_	O
investment	_	_	O
.	_	_	O
Due	_	_	O
to	_	_	O
the	_	_	O
risk	_	_	O
of	_	_	O
default	_	_	O
of	_	_	O
foreign	_	_	O
countries	_	_	O
,	_	_	O
no	_	_	B-CONST_DIR
more	_	_	I-CONST_DIR
than	_	_	I-CONST_DIR
40	_	_	B-LIMIT
%	_	_	I-LIMIT
of	_	_	O
the	_	_	O
total	_	_	O
investment	_	_	O
should	_	_	O
be	_	_	O
allocated	_	_	O
to	_	_	O
foreign	_	_	B-VAR
bonds	_	_	I-VAR
.	_	_	O
How	_	_	O
much	_	_	O
should	_	_	O
the	_	_	O
Cautious	_	_	O
Asset	_	_	O
Investment	_	_	O
allocate	_	_	O
in	_	_	O
each	_	_	O
asset	_	_	O
so	_	_	O
as	_	_	O
to	_	_	O
maximize	_	_	B-OBJ_DIR
its	_	_	O
average	_	_	B-OBJ_NAME
return	_	_	I-OBJ_NAME
?	_	_	O

A	_	_	O
flooring	_	_	O
company	_	_	O
produces	_	_	O
engineered	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-VAR
.	_	_	O
Their	_	_	O
sales	_	_	O
forecasts	_	_	O
show	_	_	O
an	_	_	O
expected	_	_	O
demand	_	_	O
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
20,000	_	_	B-LIMIT
square	_	_	O
foot	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
10,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-VAR
each	_	_	O
week	_	_	O
.	_	_	O
To	_	_	O

## Unlabeled text (2):
satisfy
a
shipping
contract
,
a
total
of
at
least
60,000
square
feet
of
flooring
much
be
shipped
each
week
.
Due
to
a
labor
shortage
issue
,
no
more
than
50,000
square
feet
of
hardwood
and
30,000
square
feet
of
vinyl
can
be
produced
weekly
.
If
a
square
foot
of
hardwood
flooring
yields
a
profit
of
$
2.5
and
a
square
foot
of
vinyl
planks
produces
a
$
3
profit
,
how
many
of
each
type
of
flooring
should
be
made
weekly
to
maximize
the
company
's
profit
?

John
has
a
300
acre
berry
farm
on
which
to
plant
blueberries
and
raspberries
.
John
has
$
10000
to
spend
on
watering
and
575
days
worth
of
labor
available
.
For
each
acre
of
blueberries
,
6
days
worth
of
labor
and
$
22
in
watering
costs
is
required
.
For
each
acre
of
raspberries
,
3
days
worth
of
labor
and
$
25
in
watering
costs
is
required
.
The
profit
per
acre
of
blueberries
is
$
56
and
the
profit
per
acre
of
raspberries
is
$
75
.
Formulate
an
LP
problem
in
order
to
maximize
profit
.

An
electronics
store
wants
to
optimize
how
many
phones
and
laptops
are
enough
to
keep
in
inventory
.
A
phone
will
earn
the
store
$
120
in
profits
,
and
a
laptop
will
earn
$
40
.
A
phone
requires
11
sq
ft
of
floor
space
,
whereas
a
laptop
requires
4
sq
ft
.
In
total
,
400
sq
ft
of
floor
## labeled text (2):
 """

In [ ]:
prompt2 = f"""
'System: the following [unlabeled text (1)] is an example of an unlabled text followed by the same text after labeling [labeled text (1)],as you can see labeled text consists of 4 columns with the first being the unlabeled text while the other three consisting of symbols and labels write [labeled text (2)] Label words as O for general words, B/I-CONST_DIR for constraint descriptions, B/I-LIMIT for numerical limits, B/I-VAR for variables, B/I-PARAM for measurable parameters, B/I-OBJ_NAME for objectives, and B-OBJ_DIR for actions toward the objective:


## unlabeled text (1):

Cautious
Asset
Investment
has
a
total
of
$
150,000
to
manage
and
decides
to
invest
it
in
money
market
fund
,
which
yields
a
2
%
return
as
well
as
in
foreign
bonds
,
which
gives
and
average
rate
of
return
of
10.2
%
.
Internal
policies
require
PAI
to
diversify
the
asset
allocation
so
that
the
minimum
investment
in
money
market
fund
is
40
%
of
the
total
investment
.
Due
to
the
risk
of
default
of
foreign
countries
,
no
more
than
40
%
of
the
total
investment
should
be
allocated
to
foreign
bonds
.
How
much
should
the
Cautious
Asset
Investment
allocate
in
each
asset
so
as
to
maximize
its
average
return
?

## labeled text (1):
Cautious	_	_	O
Asset	_	_	O
Investment	_	_	O
has	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
$	_	_	O
150,000	_	_	B-LIMIT
to	_	_	O
manage	_	_	O
and	_	_	O
decides	_	_	O
to	_	_	O
invest	_	_	O
it	_	_	O
in	_	_	O
money	_	_	B-VAR
market	_	_	I-VAR
fund	_	_	I-VAR
,	_	_	O
which	_	_	O
yields	_	_	O
a	_	_	O
2	_	_	B-PARAM
%	_	_	I-PARAM
return	_	_	B-OBJ_NAME
as	_	_	O
well	_	_	O
as	_	_	O
in	_	_	O
foreign	_	_	B-VAR
bonds	_	_	I-VAR
,	_	_	O
which	_	_	O
gives	_	_	O
and	_	_	O
average	_	_	O
rate	_	_	O
of	_	_	O
return	_	_	B-OBJ_NAME
of	_	_	O
10.2	_	_	B-PARAM
%	_	_	O
.	_	_	O
Internal	_	_	O
policies	_	_	O
require	_	_	O
PAI	_	_	O
to	_	_	O
diversify	_	_	O
the	_	_	O
asset	_	_	O
allocation	_	_	O
so	_	_	O
that	_	_	O
the	_	_	O
minimum	_	_	B-CONST_DIR
investment	_	_	O
in	_	_	O
money	_	_	B-VAR
market	_	_	I-VAR
fund	_	_	I-VAR
is	_	_	O
40	_	_	B-LIMIT
%	_	_	I-LIMIT
of	_	_	O
the	_	_	O
total	_	_	O
investment	_	_	O
.	_	_	O
Due	_	_	O
to	_	_	O
the	_	_	O
risk	_	_	O
of	_	_	O
default	_	_	O
of	_	_	O
foreign	_	_	O
countries	_	_	O
,	_	_	O
no	_	_	B-CONST_DIR
more	_	_	I-CONST_DIR
than	_	_	I-CONST_DIR
40	_	_	B-LIMIT
%	_	_	I-LIMIT
of	_	_	O
the	_	_	O
total	_	_	O
investment	_	_	O
should	_	_	O
be	_	_	O
allocated	_	_	O
to	_	_	O
foreign	_	_	B-VAR
bonds	_	_	I-VAR
.	_	_	O
How	_	_	O
much	_	_	O
should	_	_	O
the	_	_	O
Cautious	_	_	O
Asset	_	_	O
Investment	_	_	O
allocate	_	_	O
in	_	_	O
each	_	_	O
asset	_	_	O
so	_	_	O
as	_	_	O
to	_	_	O
maximize	_	_	B-OBJ_DIR
its	_	_	O
average	_	_	B-OBJ_NAME
return	_	_	I-OBJ_NAME
?	_	_	O

A	_	_	O
flooring	_	_	O
company	_	_	O
produces	_	_	O
engineered	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-VAR
.	_	_	O
Their	_	_	O
sales	_	_	O
forecasts	_	_	O
show	_	_	O
an	_	_	O
expected	_	_	O
demand	_	_	O
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
20,000	_	_	B-LIMIT
square	_	_	O
foot	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
10,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-VAR
each	_	_	O
week	_	_	O
.	_	_	O
To	_	_	O

## Unlabeled text (2):
satisfy
a
shipping
contract
,
a
total
of
at
least
60,000
square
feet
of
flooring
much
be
shipped
each
week
.
Due
to
a
labor
shortage
issue
,
no
more
than
50,000
square
feet
of
hardwood
and
30,000
square
feet
of
vinyl
can
be
produced
weekly
.
If
a
square
foot
of
hardwood
flooring
yields
a
profit
of
$
2.5
and
a
square
foot
of
vinyl
planks
produces
a
$
3
profit
,
how
many
of
each
type
of
flooring
should
be
made
weekly
to
maximize
the
company
's
profit
?

John
has
a
300
acre
berry
farm
on
which
to
plant
blueberries
and
raspberries
.
John
has
$
10000
to
spend
on
watering
and
575
days
worth
of
labor
available
.
For
each
acre
of
blueberries
,
6
days
worth
of
labor
and
$
22
in
watering
costs
is
required
.
For
each
acre
of
raspberries
,
3
days
worth
of
labor
and
$
25
in
watering
costs
is
required
.
The
profit
per
acre
of
blueberries
is
$
56
and
the
profit
per
acre
of
raspberries
is
$
75
.
Formulate
an
LP
problem
in
order
to
maximize
profit
.

An
electronics
store
wants
to
optimize
how
many
phones
and
laptops
are
enough
to
keep
in
inventory
.
A
phone
will
earn
the
store
$
120
in
profits
,
and
a
laptop
will
earn
$
40
.
A
phone
requires
1
sq
ft
of
floor
space
,
whereas
a
laptop
requires
4
sq
ft
.
In
total
,
400
sq
ft
of
floor
## labeled text (2):
"""

In [ ]:
output1 = llm(prompt)

<ipython-input-10-0bd638890937>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output1 = llm(prompt)


 A	_	_	O
flooring	_	_	B-VAR
company	_	_	B-ORG
produces	_	_	O
engineered	_	_	B-ATTR
hardwood	_	_	I-VAR
and	_	_	O
vinyl	_	_	I-VAR
planks	_	_	I-ATTR
.	_	_	O
Their	_	_	O
sales	_	_	O
forecasts	_	_	O
show	_	_	O
an	_	_	O
expected	_	_	O
demand	_	_	B-OBJ_NAME
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
20,000	_	_	B-LIMIT
square	_	_	B-ATTR
foot	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
10,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	B-ATTR
of	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-ATTR
each	_	_	O
week	_	_	B-OBJ_NAME
.	_	_	O
To	_	_	O
  satisfy	_	_	O
a	_	_	O
shipping	_	_	B-OBJ_DIR
contract	_	_	I-OBJ_DIR
,	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
60,000	_	_	B-LIMIT
square	_	_	B-ATTR
feet	_	_	B-ATTR
of	_	_	O
flooring	_	_	B-OBJ_NAME
much	_	_	O
be	_	_	O
shipped	_	_	O
each	_	_	O
week	_	_	B-OBJ_NAME
.	_	_	O
Due	_	_	O
to	_	_	O
a	_	_	O
labor	_	_	B-OBJ_DIR
shortage	_	_	I-OBJ_DIR
issue	_	_	O
,	_	_	O
no	_	_	B-CONST_DIR
more	_	_	I-CONST_DIR
than	_	_	I-CONST_DIR
50,000


llama_print_timings:        load time =   11157.39 ms
llama_print_timings:      sample time =     525.16 ms /  1024 runs   (    0.51 ms per token,  1949.89 tokens per second)
llama_print_timings: prompt eval time =   13298.95 ms /  2436 tokens (    5.46 ms per token,   183.17 tokens per second)
llama_print_timings:        eval time =   41220.95 ms /  1023 runs   (   40.29 ms per token,    24.82 tokens per second)
llama_print_timings:       total time =   60841.37 ms /  3459 tokens


In [ ]:
output2 = llm(prompt2)

Llama.generate: prefix-match hit


satisfy	_	_	O
a	_	_	O
shipping	_	_	O
contract	_	_	O
,	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
60,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
flooring	_	_	O
much	_	_	O
be	_	_	O
shipped	_	_	O
each	_	_	O
week	_	_	O
.	_	_	O
Due	_	_	O
to	_	_	O
a	_	_	O
labor	_	_	O
shortage	_	_	O
issue	_	_	O
,	_	_	O
no	_	_	B-CONST_DIR
more	_	_	I-CONST_DIR
than	_	_	I-CONST_DIR
50,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
and	_	_	_	O
30,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
vinyl	_	_	B-VAR
can	_	_	O
be	_	_	O
produced	_	_	O
weekly	_	_	O
.	_	_	O
If	_	_	O
a	_	_	O
square	_	_	O
foot	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
flooring	_	_	I-VAR
yields	_	_	O
a	_	_	O
$	_	_	B-PARAM
2.5	_	_	I-PARAM
profit	_	_	B-OBJ_NAME
,	_	_	O
and	_	_	O
a	_	_	O
square	_	_	O
foot	_	_	O
of	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-VAR
produces	_	_	O
a	_	_	O
$	_	_	B-PARAM
3	_	_	I-PARAM
profit	_	_	B-OBJ_NAME
,	_	_	O
how	_	_	O
many	_	_	O
of	_	_	O
each	_	_	O
type	_	_	O
of


llama_print_timings:        load time =   11157.39 ms
llama_print_timings:      sample time =     511.46 ms /  1024 runs   (    0.50 ms per token,  2002.11 tokens per second)
llama_print_timings: prompt eval time =    2847.01 ms /  2492 tokens (    1.14 ms per token,   875.31 tokens per second)
llama_print_timings:        eval time =   42699.24 ms /  1023 runs   (   41.74 ms per token,    23.96 tokens per second)
llama_print_timings:       total time =   51618.97 ms /  3515 tokens


# Evaluation

## First we need to check if all the provided text got labeled and remove any excess

In [ ]:
def extract_first_word(text):
    # Split the text by lines
    lines = text.splitlines()

    # Extract the first word from each line
    first_words = [line.split()[0] for line in lines if line.strip()]

    # Join the first words into a single string with newlines
    result = "\n".join(first_words)

    return result

# Example input
text = """satisfy	_	_	O
a	_	_	O
shipping	_	_	O
contract	_	_	O
,	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
60,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	O
of	_	_	O
flooring	_	_	O
much	_	_	O
be	_	_	O
shipped	_	_	O"""

# Extract and print the result
result = extract_first_word(text)
print(result)


satisfy
a
shipping
contract
,
a
total
of
at
least
60,000
square
feet
of
flooring
much
be
shipped


In [ ]:
def compare_texts(text1, text2):
    # Split the texts by line and treat each line as a word
    words1 = [word.strip() for word in text1.splitlines() if word.strip()]
    words2 = [word.strip() for word in text2.splitlines() if word.strip()]

    # Convert lists to sets to compare
    set1 = set(words1)
    set2 = set(words2)

    # Find common words
    common_words = set1.intersection(set2)

    # Calculate the percentage of common words
    total_unique_words = len(set1.union(set2))
    common_percentage = (len(common_words) / total_unique_words) * 100

    # Find words in text1 but not in text2
    words_only_in_text1 = set1 - set2

    # Find words in text2 but not in text1
    words_only_in_text2 = set2 - set1

    # Output results
    print(f"Percentage of common words: {common_percentage:.2f}%")
    print(f"Words only in the first text: {list(words_only_in_text1)}")
    print(f"Words only in the second text: {list(words_only_in_text2)}")

    return common_percentage, list(words_only_in_text1), list(words_only_in_text2)


common_percentage, missing_in_text1, missing_in_text2 = compare_texts(input_text, extract_first_word(output1))
common_percentage, missing_in_text1, missing_in_text2 = compare_texts(input_text, extract_first_word(output2))


Percentage of common words: 36.72%
Words only in the first text: ['on', '6', 'ft', '25', 'how', 'profits', '4', '400', 'enough', '75', 'laptops', 'spend', 'floor', 'space', 'electronics', 'inventory', 'requires', 'costs', 'wants', 'John', 'available', 'many', '?', 'which', 'whereas', 'For', 'berry', 'should', 'earn', 'blueberries', '10000', 'An', 'store', '40', '575', '3', "'s", 'has', 'farm', 'LP', 'Formulate', 'required', 'made', 'sq', 'the', 'problem', 'per', 'optimize', 'phone', 'laptop', 'type', 'is', 'The', 'keep', 'In', 'worth', 'plant', 'maximize', 'watering', 'are', '22', 'raspberries', 'phones', '120', 'acre', 'days', '1', 'order', '300', '56', 'will']
Words only in the second text: ['forecasts', 'Their', 'engineered', 'demand', 'expected', 'To', '20,000', 'sales', 'show', '10,000']
Percentage of common words: 54.62%
Words only in the first text: ['6', 'ft', '25', 'profits', 'an', '4', 'A', '400', 'enough', '75', 'laptops', 'spend', 'floor', 'space', 'electronics', 'inventory

In [ ]:
print(output1)

 A	_	_	O
flooring	_	_	B-VAR
company	_	_	B-ORG
produces	_	_	O
engineered	_	_	B-ATTR
hardwood	_	_	I-VAR
and	_	_	O
vinyl	_	_	I-VAR
planks	_	_	I-ATTR
.	_	_	O
Their	_	_	O
sales	_	_	O
forecasts	_	_	O
show	_	_	O
an	_	_	O
expected	_	_	O
demand	_	_	B-OBJ_NAME
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
20,000	_	_	B-LIMIT
square	_	_	B-ATTR
foot	_	_	O
of	_	_	O
hardwood	_	_	B-VAR
and	_	_	O
10,000	_	_	B-LIMIT
square	_	_	O
feet	_	_	B-ATTR
of	_	_	O
vinyl	_	_	B-VAR
planks	_	_	I-ATTR
each	_	_	O
week	_	_	B-OBJ_NAME
.	_	_	O
To	_	_	O
  satisfy	_	_	O
a	_	_	O
shipping	_	_	B-OBJ_DIR
contract	_	_	I-OBJ_DIR
,	_	_	O
a	_	_	O
total	_	_	B-CONST_DIR
of	_	_	O
at	_	_	B-CONST_DIR
least	_	_	I-CONST_DIR
60,000	_	_	B-LIMIT
square	_	_	B-ATTR
feet	_	_	B-ATTR
of	_	_	O
flooring	_	_	B-OBJ_NAME
much	_	_	O
be	_	_	O
shipped	_	_	O
each	_	_	O
week	_	_	B-OBJ_NAME
.	_	_	O
Due	_	_	O
to	_	_	O
a	_	_	O
labor	_	_	B-OBJ_DIR
shortage	_	_	I-OBJ_DIR
issue	_	_	O
,	_	_	O
no	_	_	B-CONST_DIR
more	_	_	I-CONST_DIR
than	_	_	I-CONST_DIR
50,000